### Generating team and salaries data
--------------------------------------------------------------------------
To add: description of data/data source

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [46]:
#Teams data import

teams_df = pd.read_csv("Baseball Dataset\Teams.csv")
                          
teams_df.head()

#Limiting data to years 1985 - 2015
teams_df = teams_df[teams_df['yearID']>=1985].reset_index(drop=True)

teams_df.head()

teams_df.groupby('franchID').count()

#Limiting to franchise in full sample
    #Note: The Arizona Diamondbacks, Colorado Rockies, Flordia Marlins, and Tampa Bay Devils franchises 
    #did not exist during the full study time perion (1985-2015) and are dropped from the dataset

excluded_franchID = ['ARI','COL','FLA','TBD']

teams_df = teams_df.loc[~teams_df['franchID'].isin(excluded_franchID)]

teams_df.groupby('franchID').count()


,yearID,lgID,teamID,divID,Rank,G,Ghome,W,L,DivWin,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
franchID,,,,,,,,,,,,,,,,,,,,,
ANA,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
ATL,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
BAL,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
BOS,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
CHC,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
CHW,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
CIN,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
CLE,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31
DET,31,31,31,31,31,31,31,31,31,30,...,31,31,31,31,31,31,31,31,31,31


In [12]:
#Salaries data import

salaries_df = pd.read_csv("Baseball Dataset\Salaries.csv")
                          
salaries_df.head()



,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [22]:
#Team player salaries summary statistics
team_salary_stats = salaries_df.groupby(['yearID','teamID']).agg(total_player_salaries = ('salary', 'sum'),
                                                                  med_player_salary = ('salary','median'),
                                                                  min_player_salary = ('salary','min'),
                                                                  max_player_salary = ('salary','max'),
                                                                  salary_var = ('salary','var'))

team_salary_stats.head()

total_player_salaries  med_player_salary  min_player_salary  \
yearID teamID                                                                
1985   ATL                  14807000           620833.5             120000   
       BAL                  11560712           497916.5              60000   
       BOS                  10897560           325000.0             115000   
       CAL                  14427894           493750.0              60000   
       CHA                   9846178           333333.0              60000   

               max_player_salary    salary_var  
yearID teamID                                   
1985   ATL               1625000  1.576369e+11  
       BAL               1472819  1.017152e+11  
       BOS               1075000  8.910658e+10  
       CAL               1100000  9.319582e+10  
       CHA               1242333  1.184021e+11

In [23]:
#Checking # of teams per year
team_salary_stats.groupby('yearID').nunique()

#Consider restricting our sample to original 26 teams


,total_player_salaries,med_player_salary,min_player_salary,max_player_salary,salary_var
yearID,,,,,
1985,26,24,10,23,26
1986,26,25,1,24,26
1987,26,26,2,25,26
1988,26,25,4,23,26
1989,26,26,2,25,26
1990,26,23,1,21,26
1991,26,24,6,24,26
1992,26,24,3,25,26
1993,28,25,3,25,28


In [52]:
#Merging Team and Salary Data
team_salary_df = pd.merge(teams_df, team_salary_stats, on = ('yearID','teamID'))

team_salary_df.head() 

# Export the merged data into a csv
team_salary_df.to_csv("Output/team_salary_merged.csv", index=False)
    

In [50]:
#Dataset info
team_salary_df.info() 

team_salary_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   yearID                 806 non-null    int64  
 1   lgID                   806 non-null    object 
 2   teamID                 806 non-null    object 
 3   franchID               806 non-null    object 
 4   divID                  806 non-null    object 
 5   Rank                   806 non-null    int64  
 6   G                      806 non-null    int64  
 7   Ghome                  806 non-null    float64
 8   W                      806 non-null    int64  
 9   L                      806 non-null    int64  
 10  DivWin                 780 non-null    object 
 11  WCWin                  546 non-null    object 
 12  LgWin                  780 non-null    object 
 13  WSWin                  780 non-null    object 
 14  R                      806 non-null    int64  
 15  AB    

,yearID,Rank,G,Ghome,W,L,R,AB,H,2B,...,DP,FP,attendance,BPF,PPF,total_player_salaries,med_player_salary,min_player_salary,max_player_salary,salary_var
count,806.000000,806.000000,806.000000,806.000000,806.000000,806.000000,806.000000,806.000000,806.000000,806.000000,...,806.000000,806.000000,8.060000e+02,806.000000,806.000000,8.060000e+02,8.060000e+02,806.000000,8.060000e+02,8.060000e+02
mean,2000.000000,3.146402,159.813896,79.893300,80.337469,79.426799,729.755583,5465.967742,1429.858561,274.428040,...,147.168734,0.979904,2.304224e+06,99.705955,99.678660,5.823619e+07,9.223391e+05,238972.776675,9.177996e+06,9.594521e+12
std,8.949826,1.625390,8.927735,4.572511,11.850215,11.793102,90.755537,308.460056,107.359626,34.715084,...,19.386973,0.005172,7.132378e+05,3.878482,3.862509,4.290284e+07,8.689924e+05,150893.932387,6.642174e+06,1.312066e+13
min,1985.000000,1.000000,112.000000,44.000000,43.000000,40.000000,466.000000,3856.000000,963.000000,159.000000,...,82.000000,0.970000,6.427450e+05,88.000000,88.000000,8.800000e+05,9.875000e+04,0.000000,6.000000e+05,2.523706e+10
25%,1992.000000,2.000000,162.000000,81.000000,72.000000,71.000000,666.250000,5468.000000,1377.000000,253.000000,...,134.000000,0.980000,1.776747e+06,97.000000,97.000000,2.348717e+07,3.375000e+05,109000.000000,3.550000e+06,9.858384e+11
50%,2000.000000,3.000000,162.000000,81.000000,81.000000,79.000000,729.000000,5524.000000,1435.000000,275.000000,...,147.000000,0.980000,2.258405e+06,100.000000,100.000000,4.803724e+07,5.612500e+05,200000.000000,7.915266e+06,4.102915e+12
75%,2008.000000,4.000000,162.000000,81.000000,89.000000,88.000000,790.000000,5584.750000,1495.000000,297.000000,...,160.000000,0.983000,2.829068e+06,102.000000,102.000000,8.318100e+07,1.200000e+06,390000.000000,1.350000e+07,1.356824e+13
max,2015.000000,7.000000,164.000000,84.000000,116.000000,119.000000,1009.000000,5781.000000,1684.000000,376.000000,...,204.000000,0.991000,4.298655e+06,117.000000,115.000000,2.319789e+08,5.833334e+06,514000.000000,3.300000e+07,8.588077e+13
